<a href="https://colab.research.google.com/github/krisivarga/deep_learning_HW_big/blob/Kristof_solve/Copy_of_get_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==1.15
!pip install tensorflow_estimator
!pip install bert-tensorflow

#Importing BERT modules
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

     |████████████████████████████████| 412.3 MB 25 kB/s 
     |████████████████████████████████| 503 kB 46.4 MB/s 
     |████████████████████████████████| 3.8 MB 37.8 MB/s 
     |████████████████████████████████| 50 kB 6.0 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=8ef087ce328d62ae4317099310dc94dcc6b82e95410d7daa52ee94ab2855d055
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Succes

As the first task, we download the given dataset from   http://groups.di.unipi.it/~gulli/AG_corpus_of_news_articles.html . We chose the xml format, as it seems to be better structured and has fewer invalid rows.

In [ ]:
#download file from web

import requests, zipfile, io, os

zip_file_url = "http://groups.di.unipi.it/~gulli/newsspace200.xml.bz"

filename = zip_file_url.split("/")[-1]
with open(filename, "wb") as f:
    r = requests.get(zip_file_url)
    f.write(r.content)
    
    #create data folder, decompress data

import bz2,shutil

dirName = 'data'
try:
    # Create target Directory
    os.mkdir(dirName)
    print("Directory " , dirName ,  " Created ") 
except FileExistsError:
    print("Directory " , dirName ,  " already exists")
    
    
with bz2.BZ2File("newsspace200.xml.bz") as fr, open("./data/input.xml","wb") as fw:
    shutil.copyfileobj(fr,fw)

Directory  data  Created 


After reading in the XML file, we parse it. We create 4 list for source, titles, descriptions and categories. 

In [ ]:
import xml.etree.ElementTree as ET

tree = ET.parse("./data/input.xml")
root = tree.getroot()

titles = []
categories = []
descriptions = []
sources = []


for title in tree.findall('title'):
    titles.append(title.text)
    
for category in tree.findall('category'):
        categories.append(category.text)

for description in tree.findall('description'):
    descriptions.append(description.text)

for source in tree.findall('source'):
    sources.append(source.text)


In [ ]:
import numpy as np

In [ ]:
titles = np.array(titles).T
categories = np.array(categories).T
descriptions = np.array(descriptions).T
sources = np.array(sources).T

In [ ]:
data = np.dstack((sources,titles,descriptions,categories)).reshape(len(titles),4)

In [ ]:
data.shape

(496835, 4)

In [ ]:
import pandas as pd

df = pd.DataFrame(data)
df.columns = ['source','title', 'desc', 'cat']

We enlist all categories found in the data. We can see that there are a few categories which are not really categories. These are casused by values which are shifted into the wrong column. We filter these rows out.

In [ ]:
print(f"Total unique categories are: {len(df['cat'].value_counts())}")
print(f"Count of occurance of each category:")
df['cat'].value_counts()

Total unique categories are: 17
Count of occurance of each category:


World                                                                                                                                                                                       81456
Entertainment                                                                                                                                                                               70892
Sports                                                                                                                                                                                      62163
Business                                                                                                                                                                                    56656
Top Stories                                                                                                                                                                                 56045
Sci/Tech                      

We select the top 5 categories from the data, and copy these rows into a different dataframe, as we only need these from future work.

In [ ]:
selected_cats = df['cat'].value_counts()[:5].index.tolist()
print(selected_cats)

df_selected = df.loc[df['cat'].isin(selected_cats)]

print(df_selected)

['World', 'Entertainment', 'Sports', 'Business', 'Top Stories']
                       source  ...       cat
0              Yahoo Business  ...  Business
1              Yahoo Business  ...  Business
2              Yahoo Business  ...  Business
3              Yahoo Business  ...  Business
4              Yahoo Business  ...  Business
...                       ...  ...       ...
496829  New York Times sports  ...    Sports
496830         BBC News world  ...     World
496831  New York Times sports  ...    Sports
496832         BBC News world  ...     World
496833  New York Times sports  ...    Sports

[327212 rows x 4 columns]


After selecting the top 5, we can see that the number of articles/catergories are inbalanced. As it is not decided yet, what will be the model to be used in the end, we did not try to solve this issue (yet).

In [ ]:
print(f"Total unique categories are: {len(df_selected['cat'].value_counts())}")
print(f"Count of occurance of each category:")
df_selected['cat'].value_counts()

Total unique categories are: 5
Count of occurance of each category:


World            81456
Entertainment    70892
Sports           62163
Business         56656
Top Stories      56045
Name: cat, dtype: int64

We check if there are any empty values in the dataset and drop those as well.

In [ ]:
df_selected.isnull().sum()

source       0
title        0
desc      2415
cat          0
dtype: int64

In [ ]:
df_selected = df_selected.dropna()

In [ ]:
df_selected.isnull().sum()

source    0
title     0
desc      0
cat       0
dtype: int64

In [ ]:
# Check of spaces in column headline - using enumerate
spaces = []
for i, x in enumerate(df_selected['title']):
    if type(x) == str:
        if x.isspace():
            spaces.append(i)
        
print(len(spaces), 'spaces in index: ', spaces)

0 spaces in index:  []


In [ ]:
# Check of spaces in column short desc - using itertuples
blanks = []  # start with an empty list

for i,sou,tit,desc,cat in df_selected.itertuples():  # iterate over the DataFrame
    if type(desc)==str:            # avoid NaN values
        if desc.isspace():         # test 'review' for whitespace
            blanks.append(i)     # add matching index numbers to the list
        
print(len(blanks), 'blanks: ', blanks)

0 blanks:  []


Although it is part of the database, we are not going to use the 'source' field. When we print the value_counts for that field joined with the categories, it's obvious that the category can be found in the 'source' most of the time, which defeats the point of this homework.

In [ ]:
print(df_selected['source'].value_counts())
print(df_selected.value_counts(subset=['source','cat'])[:30])
print(df_selected.value_counts(subset=['source','cat'])[100:130])

Reuters                        15935
Yahoo World                    11193
RedNova general                 7855
Reuters Business                6897
Yahoo Sports                    6850
                               ...  
The Auburn Plainsman Online        1
Tahlequah Daily Press              1
European Space Agency (ESA)        1
Skagit Valley Herald               1
BeaverFootball.com                 1
Name: source, Length: 3053, dtype: int64
source                      cat          
Yahoo World                 World            11193
RedNova general             World             7855
Reuters Business            Business          6897
Yahoo Sports                Sports            6850
Yahoo Entertainment         Entertainment     6741
Reuters World               World             6106
BBC News world              World             6038
Yahoo Politics              World             4586
New York Times sports       Sports            4361
Reuters                     Top Stories       3710


Next, we define tokenization and vocabulary building. We're using the WordPiece based BERT tokenizer from huggingface.co. We also tried the SentencePie based XLNetTokenizer, but the results weren't as good as with the BERT tokenizer. The methods that are defined here will be applied to every line of the dataframe.  
  
We decided on one method, which tokenizes the title and the description and after that, it removes the tokens that contain non-alphabet characters. We do this because some descriptions contain obviously unwanted characters, like '\\' (and as we can later see from the vocabulary "39" exists more than 149000 times in the text). We also remove any token that is less than 3 characters. We build the vocabualry from these tokens.  
    
The other method uses the encode_plus function which automatically encodes the tokens too. In the function parameters we set the maximum length to 512, set the padding to extend to the max length, added special tokens and didn't take the attention mask. Depending on what modles we use, we may later need to change those settings (for example if we use BERT or XLNet models). We created a vocabulary based on the tokens too.  
  
The vocabularies can contain data multiple times, because we can see the number of appearance of tokens in the text that way if we use the value_counts function.

In [ ]:
!pip install sentencepiece
!pip install transformers
from transformers import XLNetTokenizer
from transformers import BertTokenizer
import re

regex = re.compile('^[a-zA-Z]{3,}$')

xltokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased")

btokenizer = BertTokenizer.from_pretrained("bert-base-uncased")



     |████████████████████████████████| 1.2 MB 28.7 MB/s 
     |████████████████████████████████| 3.1 MB 28.2 MB/s 
     |████████████████████████████████| 895 kB 45.2 MB/s 
     |████████████████████████████████| 3.3 MB 33.3 MB/s 
     |████████████████████████████████| 596 kB 55.2 MB/s 
     |████████████████████████████████| 61 kB 448 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
def tr(bt,reg,title,sz):
  a = bt.tokenize(title + " " + sz)
  r = []
  for s in a:
    if reg.match(s) is not None:     
      r.append(s)      
  return r

df_selected['berttoken'] = df_selected.apply(lambda row: tr(btokenizer, regex, str(row['title']), str(row['desc'])), axis=1)
df_selected

,source,title,desc,cat,berttoken
0,Yahoo Business,Wall St. Pullback Reflects Tech Blowout (Reuters),"Reuters - Wall Street's long-playing drama,\""W...",Business,"[wall, pull, reflects, tech, blow, reuters, re..."
1,Yahoo Business,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli...",Business,"[wall, bears, claw, back, into, the, black, re..."
2,Yahoo Business,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...,Business,"[carly, looks, toward, commercial, aerospace, ..."
3,Yahoo Business,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...,Business,"[oil, and, economy, cloud, stocks, outlook, re..."
4,Yahoo Business,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...,Business,"[iraq, halt, oil, exports, from, main, souther..."
...,...,...,...,...,...
496829,New York Times sports,High on priority list: Home improvement,Doc Rivers knows any postseason plans hinge on...,Sports,"[high, priority, list, home, improvement, doc,..."
496830,BBC News world,Compromise seals climate meeting,A climate conference overcomes last-minute obj...,World,"[compromise, seals, climate, meeting, climate,..."
496831,New York Times sports,Eisley enjoying his point of view,Howard Eisley has fond memories of Boston. He ...,Sports,"[enjoying, his, point, view, howard, has, fond..."
496832,BBC News world,Iraqi judges quiz 'Chemical Ali',Ali Hassan al-Majid - widely known as Chemical...,World,"[iraqi, judges, quiz, chemical, ali, ali, hass..."


In [ ]:
def bertenc(bt,title,sz):
  text_to_encode = title + " " + sz
  a = bt.encode_plus(
            text_to_encode, 
            max_length= 512, 
            add_special_tokens=True,
            return_token_type_ids=False, 
            padding="max_length",
            truncation = True,
            return_attention_mask=False
        )
  ids = a['input_ids']  
  return ids

df_selected['bertencodeplus'] = df_selected.apply(lambda row: bertenc(btokenizer, str(row['title']), str(row['desc'])), axis=1)

df_selected.to_csv("df_selected_export.csv", index=False, sep="\t")
df_selected

,source,title,desc,cat,berttoken,bertencodeplus
0,Yahoo Business,Wall St. Pullback Reflects Tech Blowout (Reuters),"Reuters - Wall Street's long-playing drama,\""W...",Business,"[wall, pull, reflects, tech, blow, reuters, re...","[101, 2813, 2358, 1012, 4139, 5963, 11138, 662..."
1,Yahoo Business,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli...",Business,"[wall, bears, claw, back, into, the, black, re...","[101, 2813, 2358, 1012, 6468, 15020, 2067, 204..."
2,Yahoo Business,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...,Business,"[carly, looks, toward, commercial, aerospace, ...","[101, 18431, 2571, 3504, 2646, 3293, 13395, 10..."
3,Yahoo Business,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...,Business,"[oil, and, economy, cloud, stocks, outlook, re...","[101, 3514, 1998, 4610, 6112, 15768, 1005, 176..."
4,Yahoo Business,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...,Business,"[iraq, halt, oil, exports, from, main, souther...","[101, 5712, 9190, 2015, 3514, 14338, 2013, 236..."
...,...,...,...,...,...,...
496829,New York Times sports,High on priority list: Home improvement,Doc Rivers knows any postseason plans hinge on...,Sports,"[high, priority, list, home, improvement, doc,...","[101, 2152, 2006, 9470, 2862, 1024, 2188, 7620..."
496830,BBC News world,Compromise seals climate meeting,A climate conference overcomes last-minute obj...,World,"[compromise, seals, climate, meeting, climate,...","[101, 12014, 13945, 4785, 3116, 1037, 4785, 30..."
496831,New York Times sports,Eisley enjoying his point of view,Howard Eisley has fond memories of Boston. He ...,Sports,"[enjoying, his, point, view, howard, has, fond...","[101, 1041, 2483, 3051, 9107, 2010, 2391, 1997..."
496832,BBC News world,Iraqi judges quiz 'Chemical Ali',Ali Hassan al-Majid - widely known as Chemical...,World,"[iraqi, judges, quiz, chemical, ali, ali, hass...","[101, 8956, 6794, 19461, 1005, 5072, 4862, 100..."


Here we can see the result of the encoded vocabulary with the token values.

In [ ]:
bertencidvoc_dict = {}
bertvoc_dict = {}
def dict_creater(arr):
  for a in arr:
    if a != 0:
      if a in bertencidvoc_dict:
        count = bertencidvoc_dict[a] + 1
        bertencidvoc_dict.update({a:count})
      else:
        bertencidvoc_dict.update({a:1})
  return

def dict_creater2(arr):
  for a in arr:
    if a != 0:
      if a in bertvoc_dict:
        count = bertvoc_dict[a] + 1
        bertvoc_dict.update({a:count})
      else:
        bertvoc_dict.update({a:1})
  return

df_selected.apply(lambda row: dict_creater(row['bertencodeplus']), axis=1)

df_selected.apply(lambda row: dict_creater2(row['berttoken']), axis=1)

bertvoc_dict

{'wall': 3388,
 'pull': 1820,
 'reflects': 134,
 'tech': 1682,
 'blow': 1375,
 'reuters': 39445,
 'street': 3845,
 'long': 6772,
 'playing': 1529,
 'drama': 900,
 'waiting': 835,
 'for': 133895,
 'google': 2773,
 'about': 17594,
 'reach': 2367,
 'its': 43224,
 'final': 8332,
 'act': 1249,
 'but': 23801,
 'stock': 4308,
 'market': 6782,
 'debut': 1434,
 'ending': 1714,
 'more': 23353,
 'nostalgia': 30,
 'event': 1675,
 'than': 17414,
 'the': 570053,
 'catalyst': 27,
 'new': 50937,
 'era': 1154,
 'bears': 783,
 'claw': 42,
 'back': 12031,
 'into': 17641,
 'black': 2809,
 'short': 2838,
 'sellers': 76,
 'band': 1291,
 'ultra': 120,
 'are': 23633,
 'seeing': 321,
 'green': 2510,
 'again': 4971,
 'carly': 126,
 'looks': 1331,
 'toward': 2296,
 'commercial': 987,
 'aerospace': 174,
 'private': 1859,
 'investment': 1591,
 'firm': 3156,
 'group': 12423,
 'which': 11381,
 'has': 51516,
 'reputation': 369,
 'making': 3040,
 'well': 3024,
 'timed': 71,
 'and': 184938,
 'occasionally': 50,
 'contr

We encoded the categories with numbers, so they're easier to work with.

In [ ]:
i = 0
category_dictionary = {}
cat_rev_dict= {}
for s in selected_cats:
  category_dictionary.update({i:s})
  cat_rev_dict.update({s:i})
  i = i + 1
print(category_dictionary)

df_selected['cat_enc'] = df_selected.apply(lambda row: cat_rev_dict[str(row['cat'])] , axis=1)
df_selected['join'] = df_selected['title']+" "+df_selected['desc']


{0: 'World', 1: 'Entertainment', 2: 'Sports', 3: 'Business', 4: 'Top Stories'}


,source,title,desc,cat,berttoken,bertencodeplus,cat_enc
0,Yahoo Business,Wall St. Pullback Reflects Tech Blowout (Reuters),"Reuters - Wall Street's long-playing drama,\""W...",Business,"[wall, pull, reflects, tech, blow, reuters, re...","[101, 2813, 2358, 1012, 4139, 5963, 11138, 662...",3
1,Yahoo Business,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli...",Business,"[wall, bears, claw, back, into, the, black, re...","[101, 2813, 2358, 1012, 6468, 15020, 2067, 204...",3
2,Yahoo Business,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...,Business,"[carly, looks, toward, commercial, aerospace, ...","[101, 18431, 2571, 3504, 2646, 3293, 13395, 10...",3
3,Yahoo Business,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...,Business,"[oil, and, economy, cloud, stocks, outlook, re...","[101, 3514, 1998, 4610, 6112, 15768, 1005, 176...",3
4,Yahoo Business,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...,Business,"[iraq, halt, oil, exports, from, main, souther...","[101, 5712, 9190, 2015, 3514, 14338, 2013, 236...",3
...,...,...,...,...,...,...,...
496829,New York Times sports,High on priority list: Home improvement,Doc Rivers knows any postseason plans hinge on...,Sports,"[high, priority, list, home, improvement, doc,...","[101, 2152, 2006, 9470, 2862, 1024, 2188, 7620...",2
496830,BBC News world,Compromise seals climate meeting,A climate conference overcomes last-minute obj...,World,"[compromise, seals, climate, meeting, climate,...","[101, 12014, 13945, 4785, 3116, 1037, 4785, 30...",0
496831,New York Times sports,Eisley enjoying his point of view,Howard Eisley has fond memories of Boston. He ...,Sports,"[enjoying, his, point, view, howard, has, fond...","[101, 1041, 2483, 3051, 9107, 2010, 2391, 1997...",2
496832,BBC News world,Iraqi judges quiz 'Chemical Ali',Ali Hassan al-Majid - widely known as Chemical...,World,"[iraqi, judges, quiz, chemical, ali, ali, hass...","[101, 8956, 6794, 19461, 1005, 5072, 4862, 100...",0


In [ ]:
df_selected['join'] = df_selected['title']+" "+df_selected['desc']
df_selected_sel = df_selected[["join","cat_enc"]]

In [ ]:
df_selected_sel

,join,cat_enc
0,Wall St. Pullback Reflects Tech Blowout (Reute...,3
1,Wall St. Bears Claw Back Into the Black (Reute...,3
2,Carlyle Looks Toward Commercial Aerospace (Reu...,3
3,Oil and Economy Cloud Stocks' Outlook (Reuters...,3
4,Iraq Halts Oil Exports from Main Southern Pipe...,3
...,...,...
496829,High on priority list: Home improvement Doc Ri...,2
496830,Compromise seals climate meeting A climate con...,0
496831,Eisley enjoying his point of view Howard Eisle...,2
496832,Iraqi judges quiz 'Chemical Ali' Ali Hassan al...,0


In [ ]:
training_data = df_selected_sel.sample(frac=0.8, random_state=25)
testing_data_df = df_selected_sel.drop(training_data.index)

print(f"No. of training examples: {training_data.shape[0]}")

testing_data = testing_data_df.sample(frac=0.5, random_state=25)
valid_data = testing_data_df.drop(testing_data.index)
print(f"No. of testing examples: {testing_data.shape[0]}")
print(f"No. of testing examples: {valid_data.shape[0]}")

No. of training examples: 259838
No. of testing examples: 32480
No. of testing examples: 32479


In [ ]:
print("Training Set Shape :", training_data.shape)
print("Validation Set Shape :", valid_data.shape)
# print("Test Set Shape :", test.shape)

Training Set Shape : (259838, 2)
Validation Set Shape : (32479, 2)


In [ ]:
DATA_COLUMN = 'join'
LABEL_COLUMN = 'cat_enc'
# The list containing all the classes (train['SECTION'].unique())
label_list = [x for x in np.unique(training_data.cat_enc)]
label_list

[0, 1, 2, 3, 4]

In [ ]:
def get_split(text1):
  l_total = []
  l_parcial = []
  if len(text1.split())//150 >0:
    n = len(text1.split())//150
  else: 
    n = 1
  for w in range(n):
    if w == 0:
      l_parcial = text1.split()[:200]
      l_total.append(" ".join(l_parcial))
    else:
      l_parcial = text1.split()[w*150:w*150 + 200]
      l_total.append(" ".join(l_parcial))
  return l_total

In [ ]:
training_data['text_split'] = training_data[DATA_COLUMN].apply(get_split)
training_data.head()

,join,cat_enc,text_split
147940,Gold miners push JSE index lower Sharp falls i...,3,[Gold miners push JSE index lower Sharp falls ...
216357,"Ellis, Bohn share lead in Las Vegas Danny Elli...",2,"[Ellis, Bohn share lead in Las Vegas Danny Ell..."
67578,Force not planned to end Russia siege-local le...,4,[Force not planned to end Russia siege-local l...
329618,Experts: Even provisional ballots couldn #39;t...,1,[Experts: Even provisional ballots couldn #39;...
66538,"Malaysia Rebel Politician Anwar Free, Urges Re...",0,"[Malaysia Rebel Politician Anwar Free, Urges R..."


In [ ]:
testing_data['text_split'] = testing_data[DATA_COLUMN].apply(get_split)
testing_data.head()

,join,cat_enc,text_split
455827,"5 Pacers, 5 Fans Charged in Detroit Brawl Indi...",1,"[5 Pacers, 5 Fans Charged in Detroit Brawl Ind..."
379971,Israel #39;s Sharon Says Could Coordinate Gaza...,0,[Israel #39;s Sharon Says Could Coordinate Gaz...
309810,A little red wine may help to ward off cancer ...,1,[A little red wine may help to ward off cancer...
358810,Across the United States NEW YORK -- The mayor...,1,[Across the United States NEW YORK -- The mayo...
412218,Freed UN hostages meet Karzai Three UN workers...,0,[Freed UN hostages meet Karzai Three UN worker...


In [ ]:
valid_data['text_split'] = valid_data[DATA_COLUMN].apply(get_split)
valid_data.head()

,join,cat_enc,text_split
49,"Google IPO: Type in 'confusing,' 'secrecy' I'v...",3,"[Google IPO: Type in 'confusing,' 'secrecy' I'..."
87,Coming to The Rescue Got a unique problem? Not...,3,[Coming to The Rescue Got a unique problem? No...
102,"IT alligator tales I grew up in New York, wher...",3,"[IT alligator tales I grew up in New York, whe..."
114,FCC Pulls Blinds on Wireless Data New rules wi...,3,[FCC Pulls Blinds on Wireless Data New rules w...
118,Paid Search Growth May Slow A new Internet adv...,3,[Paid Search Growth May Slow A new Internet ad...


In [ ]:
train_l = []
label_l = []
index_l =[]
for idx,row in training_data.iterrows():
  for l in row['text_split']:
    train_l.append(l)
    label_l.append(row['cat_enc'])
    index_l.append(idx)
len(train_l), len(label_l), len(index_l)

(259852, 259852, 259852)

In [ ]:
val_l = []
val_label_l = []
val_index_l = []
for idx,row in valid_data.iterrows():
  for l in row['text_split']:
    val_l.append(l)
    val_label_l.append(row['cat_enc'])
    val_index_l.append(idx)
len(val_l), len(val_label_l), len(val_index_l)

(32480, 32480, 32480)

In [ ]:
test_l = []
test_label_l = []
test_index_l = []
for idx,row in testing_data.iterrows():
  for l in row['text_split']:
    test_l.append(l)
    test_label_l.append(row['cat_enc'])
    test_index_l.append(idx)
len(test_l), len(test_label_l), len(test_index_l)

(32480, 32480, 32480)

In [ ]:
train_df = pd.DataFrame({DATA_COLUMN:train_l, LABEL_COLUMN:label_l})
train_df.head()

,join,cat_enc
0,Gold miners push JSE index lower Sharp falls i...,3
1,"Ellis, Bohn share lead in Las Vegas Danny Elli...",2
2,Force not planned to end Russia siege-local le...,4
3,Experts: Even provisional ballots couldn #39;t...,1
4,"Malaysia Rebel Politician Anwar Free, Urges Re...",0


In [ ]:
val_df = pd.DataFrame({DATA_COLUMN:val_l, LABEL_COLUMN:val_label_l})
val_df.head()

,join,cat_enc
0,"Google IPO: Type in 'confusing,' 'secrecy' I'v...",3
1,Coming to The Rescue Got a unique problem? Not...,3
2,"IT alligator tales I grew up in New York, wher...",3
3,FCC Pulls Blinds on Wireless Data New rules wi...,3
4,Paid Search Growth May Slow A new Internet adv...,3


In [ ]:
test_df = pd.DataFrame({DATA_COLUMN:test_l, LABEL_COLUMN:test_label_l})
test_df.head()

,join,cat_enc
0,"5 Pacers, 5 Fans Charged in Detroit Brawl Indi...",1
1,Israel #39;s Sharon Says Could Coordinate Gaza...,0
2,A little red wine may help to ward off cancer ...,1
3,Across the United States NEW YORK -- The mayor...,1
4,Freed UN hostages meet Karzai Three UN workers...,0


In [ ]:
train_InputExamples = training_data.apply(lambda x: bert.run_classifier.InputExample(guid=None,
                                                                   text_a = x["join"], 
                                                                   text_b = None, 
                                                                   label = x["cat_enc"]), axis = 1)

val_InputExamples = valid_data.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x["join"], 
                                                                   text_b = None, 
                                                                   label = x["cat_enc"]), axis = 1)


test_InputExamples = testing_data.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x["join"], 
                                                                   text_b = None, 
                                                                   label = x["cat_enc"]), axis = 1)

In [ ]:
import tensorflow as tf


# Set the output directory for saving model file
OUTPUT_DIR = '/bert_news_category'

#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = False #@param {type:"boolean"}

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    pass

tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

***** Model output directory: /bert_news_category *****


In [ ]:
print("Row 0 - guid of training set : ", train_InputExamples.iloc[0].guid)
print("\n__________\nRow 0 - text_a of training set : ", train_InputExamples.iloc[0].text_a)
print("\n__________\nRow 0 - text_b of training set : ", train_InputExamples.iloc[0].text_b)
print("\n__________\nRow 0 - label of training set : ", train_InputExamples.iloc[0].label)

Row 0 - guid of training set :  None

__________
Row 0 - text_a of training set :  Gold miners push JSE index lower Sharp falls in Anglo American and miner Gold Fields pushed South Africa #39;s benchmark JSE Top-40 index 1.7 percent lower on Monday.

__________
Row 0 - text_b of training set :  None

__________
Row 0 - label of training set :  3


In [ ]:
import tensorflow_hub as hub

In [ ]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:absl:Using /tmp/tfhub_modules to cache modules.
INFO:absl:Downloading TF-Hub Module 'https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1'.
INFO:absl:Downloaded https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1, Total size: 421.86MB
INFO:absl:Downloaded TF-Hub Module 'https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1'.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
len(tokenizer.vocab.keys())

30522

In [ ]:
#Here is what the tokenised sample of the first training set observation looks like
print(tokenizer.tokenize(train_InputExamples.iloc[0].text_a))

UnparsedFlagAccessError: ignored

In [ ]:
# Transform the test data before predicting
X_test_cv = cv.transform(X_test)

In [ ]:
# Form a prediction set
predictions = clf.predict(X_test_cv)

In [ ]:
import sklearn.metrics as metrics
# Report the confusion matrix
print(metrics.confusion_matrix(y_test,predictions))
# Print a classification report
print(metrics.classification_report(y_test,predictions))
# Print the overall accuracy
print(metrics.accuracy_score(y_test,predictions))

[[12531  1478   497   825  1025]
 [ 3709  4474  1495  1491  2915]
 [  255   627 10905   100   483]
 [  520   702   118  9092   466]
 [ 3515  3489  1414  1288  1546]]
              precision    recall  f1-score   support

           0       0.61      0.77      0.68     16356
           1       0.42      0.32      0.36     14084
           2       0.76      0.88      0.81     12370
           3       0.71      0.83      0.77     10898
           4       0.24      0.14      0.17     11252

    accuracy                           0.59     64960
   macro avg       0.55      0.59      0.56     64960
weighted avg       0.55      0.59      0.56     64960

0.593411330049261


In [ ]:
y_train.value_counts()

0    48831
1    42547
2    37328
4    33523
3    32648
Name: cat_enc, dtype: int64

In [ ]:
#remove all files 
import shutil
os.remove('newsspace200.xml.bz')
os.remove('df_selected_export.csv')
shutil.rmtree('data')